In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("data.csv")

In [3]:
dataset.head(3)

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1


In [4]:
dataset.isnull().sum()

URLs         0
Headline     0
Body        21
Label        0
dtype: int64

In [5]:
dataset.drop(columns="URLs",axis=1,inplace=True)
dataset.drop(columns="Headline",axis=1,inplace=True)

In [6]:
dataset.dropna(inplace=True)

In [7]:
dataset.isnull().sum()

Body     0
Label    0
dtype: int64

In [8]:
dataset=dataset.sample(frac=1)

In [9]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import string

In [10]:
def preprocessing(text):
    """
    Function that does all the steps of Natural Language Processing.
    
    """
    
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    cleaned_text = " ".join(tokens)
   
    return cleaned_text

In [11]:
dataset["Body"]=dataset["Body"].apply(preprocessing)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer 

In [13]:
vectorizer=CountVectorizer()

In [14]:
X=vectorizer.fit_transform(dataset["Body"])

In [15]:
Y=dataset[["Label"]]

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,random_state=42,test_size=0.2)

In [18]:
from sklearn.naive_bayes import MultinomialNB

In [19]:
mnb=MultinomialNB()
mnb.fit(x_train,y_train)

C:\Users\Suchana\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()

In [20]:
mnb.score(x_test,y_test)*100,mnb.score(x_train,y_train)*100

(95.48872180451127, 98.55799373040752)

In [21]:
def user_input(user_text):
    """
    Function to Predict User Input
    """
      
    cleaned_text = preprocessing(user_text)  # Preprocess it
    transformed_text = vectorizer.transform([cleaned_text])  # Convert to numerical format
    prediction = mnb.predict(transformed_text)# Predict
    
    if prediction==1:
        print("Not a fake News")
    else:
        print("Fake news")


In [22]:
news=input("Enter news>>>\n")

Enter news>>>
 Neil Robertson made history by winning the 2025 World Grand Prix in Hong Kong with a 10-0 demolition of Stuart Bingham, earning a £180,000 prize. Robertson became the first player to achieve two whitewashes in multi-session finals, with his previous 9-0 win at the 2020 European Masters. He is only the second player to achieve such a whitewash in a best-of-19 frame final since Steven Davis in 1989. Robertson's impressive performance included breaks of 87, 104, 71, 59, 72, 78, and 66. An emotional Bingham, who faced personal tragedies with the recent deaths of his coach Joe Lazarus and Uncle Ray, acknowledged missed opportunities but praised Robertson's remarkable play. Robertson expressed gratitude for the support he received in Hong Kong and noted his fortune during the match, admitting the score could have been closer. This victory marks Robertson's second title of the season, following his win at the English Open. 


In [23]:
user_input(news)

Not a fake News
